In [1]:
import datacube_stats
import matplotlib
import os

import numpy as np
import rasterio.merge

from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles

import yaml
from datacube_stats import StatsApp
from datacube import Datacube
from datacube.helpers import write_geotiff

In [2]:
bands = [('count_wet','int'),('count_clear','int'),('frequency','float')]

filename='stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_'
overview_resampling = 'nearest'
target_crs='EPSG:3577'

result_name = 'wofs_summary'

config_yaml = """
sources:
  - product: ls8_usgs_wofs_scene
    measurements: [water]
    group_by: solar_day
    mask_nodata: False
    fuse_func: digitalearthau.utils.wofs_fuser

date_ranges:
    start_date: 2018-01-01
    end_date: 2018-12-31
storage:
    # this driver enables in-memory computation
    driver: xarray
    crs: EPSG:3577
    tile_size:
        x: 160000.0
        y: 160000.0
    resolution:
        x: 30
        y: -30
    chunking:
        x: 200
        y: 200
        time: 1
    dimension_order: [time, y, x]
computation:
    chunking:
        x: 800
        y: 800

input_region:
    "geometry": {
        "type": "Polygon",
        "coordinates": [ [
            [ 146.1199562, -38.536405 ],
            [ 148.8113995, -38.536405 ],
            [ 148.8113995, -36.4012989 ],
            [ 146.1199562, -36.4012989 ],
            [ 146.1199562, -38.536405 ]
          ] ]
      }

output_products:
  - name: wofs_summary
    statistic: wofs_summary
    product_type: wofs_statistical_summary
"""

In [3]:
def create_cog(input, output, overview_resampling, bidx):
    cogeo_profile = 'deflate'
    nodata = -1
    overview_level = 6
    overview_resampling = overview_resampling
    threads = 8
    
    output_profile = cog_profiles.get(cogeo_profile)
    output_profile.update(dict(BIGTIFF=os.environ.get("BIGTIFF", "IF_SAFER")))
    
    block_size = min(
        int(output_profile["blockxsize"]), int(output_profile["blockysize"])
    )

    config = dict(
        NUM_THREADS=threads,
        GDAL_TIFF_INTERNAL_MASK=os.environ.get("GDAL_TIFF_INTERNAL_MASK", True),
        GDAL_TIFF_OVR_BLOCKSIZE=os.environ.get("GDAL_TIFF_OVR_BLOCKSIZE", block_size),
    )
    print('creating '+output)
    
    
    cog_translate(
        src_path=input,
        dst_path=output,
        dst_kwargs=output_profile,
        indexes=bidx,
        nodata=nodata,
        web_optimized=False,
        add_mask=False,
        overview_level=overview_level,
        overview_resampling=overview_resampling,
        config=config,
        quiet=False
    )
    print('created '+output)

In [4]:
def write_stats_band_tile(stats, key, filename, target_crs):
    slim_dataset = stats[[key]].squeeze()  # create a one band dataset
    attrs = slim_dataset[key].attrs.copy()  # To get nodata in
    slim_dataset.attrs['crs'] = target_crs
    output_filename = filename+key+'_'+str(tile_no)+'_TEMP'+'.tif'
    write_geotiff(output_filename, slim_dataset, profile_override=attrs)
    return output_filename

In [5]:
def combine_tiles(tiles_locations,band,filename):
    key = band[0]
    inbound_dtype = band[1]
    files = tiles_locations[key]
    output_filename = filename+key+'_TEMP'+'.tif'
    sources = [rasterio.open(path) for path in files]
    dest, out_transform = rasterio.merge.merge(sources)

    with rasterio.open(files[0]) as src:
        profile = src.profile
    
    profile['transform'] = out_transform
    profile['width'] = len(dest[0][0])
    profile['height'] = len(dest[0])
    
    with rasterio.open(output_filename, 'w', **profile) as dst:
        if inbound_dtype == 'int':
            dst.write(dest.astype(rasterio.int16))
        elif inbound_dtype == 'float':
            dst.write(dest.astype(rasterio.float32))
    
    [os.remove(path) for path in files]
    return output_filename

In [6]:
config = yaml.load(config_yaml)

dc = Datacube()
app = StatsApp(config, dc.index)

tasks = app.generate_tasks(dc.index)

tile_no = 0
stats_tiles_locations = {}
stats = None
for band in bands:
    stats_tiles_locations[band[0]] = []

for task in tasks:
    stats = app.execute_task(task).result[result_name]
    for band in bands:
        stats_tiles_locations[band[0]].append(write_stats_band_tile(stats,band[0],filename,target_crs))
    tile_no = tile_no + 1
    del stats
    
for band in bands:
    uncogged_output_file = combine_tiles(stats_tiles_locations,band,filename)
    target_filename = filename+band[0]+'.tif'
    #as we have created this bands separately, band index (bidx) is always 0
    bidx = 0
    create_cog(uncogged_output_file, target_filename, overview_resampling, bidx)
    os.remove(uncogged_output_file)

creating stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_count_wet.tif


Reading input: stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_count_wet_TEMP.tif

Adding overviews...
Updating dataset tags...
Writing output to: stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_count_wet.tif


created stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_count_wet.tif
creating stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_count_clear.tif


Reading input: stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_count_clear_TEMP.tif

Adding overviews...
Updating dataset tags...
Writing output to: stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_count_clear.tif


created stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_count_clear.tif
creating stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_frequency.tif


Reading input: stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_frequency_TEMP.tif

Adding overviews...
Updating dataset tags...
Writing output to: stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_frequency.tif


created stats/ls8_wofs_victoria_3577_2018-01-01_2018-12-31_frequency.tif
